In [17]:
def dox():
    i = Isaac(noblock=False)
    c = set()
    while True:
        n = i.rand(2**32)
        if n in c: 
            return len(c),n
        c.add(n)

In [24]:
from math import log,ceil
from bloom_filter import BloomFilter
from random import getrandbits

import cProfile

def dNot(error_rate):
    return ceil(-log(error_rate,2))

def do(exp=32,error_rate=0.001):
    y = dNot(error_rate)
    c = (exp+1)/2 + log(log(y+1,2),2)
    b1 =  BloomFilter(max_elements=2**c, error_rate=2**-y)
    while True:
        n = getrandbits(exp)
        if n in b1: return n
        b1.add(n)

a = cProfile.run('do()')
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()

         4315411 function calls in 8.689 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.254    0.254    8.689    8.689 <ipython-input-24-87a63533281a>:10(do)
        1    0.000    0.000    0.000    0.000 <ipython-input-24-87a63533281a>:7(dNot)
        1    0.000    0.000    8.689    8.689 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 bloom_filter.py:389(__init__)
    69369    0.156    0.000    0.205    0.000 bloom_filter.py:394(is_set)
   706112    1.492    0.000    1.971    0.000 bloom_filter.py:400(set)
   256770    1.231    0.000    1.231    0.000 bloom_filter.py:449(simple_hash)
   128385    0.197    0.000    0.830    0.000 bloom_filter.py:457(hash1)
   128385    0.186    0.000    0.784    0.000 bloom_filter.py:462(hash2)
   903866    2.501    0.000    4.679    0.000 bloom_filter.py:467(get_bitno_lin_comb)
        1    0.000    0.000    0.000    0.000 bloom_filter.py:507(__init__)
  

In [37]:
import cProfile, pstats, io
pr = cProfile.Profile()
pr.enable()
do()
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).strip_dirs().sort_stats('tottime')
ps.print_stats()
print(s.getvalue())

         6377666 function calls in 11.496 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1337230    3.324    0.000    6.181    0.000 bloom_filter.py:467(get_bitno_lin_comb)
  1041689    2.020    0.000    2.622    0.000 bloom_filter.py:400(set)
   378798    1.625    0.000    1.625    0.000 bloom_filter.py:449(simple_hash)
    94699    1.603    0.000    8.002    0.000 bloom_filter.py:557(add)
  1904655    1.006    0.000    1.006    0.000 {built-in method builtins.divmod}
    94700    0.412    0.000    3.089    0.000 bloom_filter.py:588(__contains__)
        1    0.299    0.299   11.495   11.495 <ipython-input-24-87a63533281a>:10(do)
   189399    0.243    0.000    1.089    0.000 bloom_filter.py:457(hash1)
   189399    0.241    0.000    1.019    0.000 bloom_filter.py:462(hash2)
   756824    0.235    0.000    0.235    0.000 {method 'append' of 'list' objects}
   106142    0.209    0.000    0.273    0.000 bloom_filter.py:394(i

In [34]:
import multiprocessing

multiprocessing.cpu_count()

4

In [13]:
import random as prng
random = prng.SystemRandom()

def test():
    k = 100
    hash_bits = 16 # Memory use should be O(k * m * log m) with m = 2**hash_bits
    stream_bits = 32
    print('start')
    
    def create_hash(numbits, p = (1<<65)-493): # I *think* you need p > 2**stream_bits. Also, p needs to be prime, have fun!
        a = random.randint(1, p - 1)
        b = random.randint(0, p - 1)
        mask = (1 << numbits) - 1
    
        def hash(x):
            return ((a * x + b) % p) & mask
    
        return hash
    
    hasher_list = [create_hash(hash_bits) for i in range(k)]
    set_list = [set() for i in range(k)]
    
    fullcoll = None
    n = 0
    while fullcoll is None:
        curr_num = prng.getrandbits(stream_bits)
        #curr_num = prng.rand((1<<stream_bits) - 1)
        hash_list = [hsh(curr_num) for hsh in hasher_list]
        in_list = [(hash_list[i] in set_list[i]) for i in range(k)]
        if all(in_list):
            fullcoll = curr_num
        else:
            for i in range(k):
                set_list[i].add(hash_list[i])
            n += 1
    
    print('first full collision: {}'.format(fullcoll))
    coll_prob = (1. - (((1 << hash_bits)-1) / (1 << hash_bits))**n) ** k
    unsize = 1 << stream_bits
    firstrep_prob = n / unsize
    for i in range(n + 1):
        firstrep_prob *= (unsize - i) / unsize
    true_prob = firstrep_prob / (firstrep_prob + coll_prob)
    print('probability that this is first repeat = {}'.format(true_prob))
    print('this is output {}'.format(n + 1))
    
%timeit -n1 -r1 test()

start
first full collision: 4068891925
probability that this is first repeat = 0.9999999361134425
this is output 91781
1 loop, best of 1: 24.3 s per loop
